# Lab 10 — Classes

### HỌ VÀ TÊN: <# NGUYEN MINH TU>

### MSSV: <25021985>

#### Hướng dẫn
- Ghi đầy đủ Họ và tên (ở dạng IN HOA) và Mã số sinh viên ở 2 block chỉ định;
- Viết phần bài làm của từng Problem vào block code python tương ứng, có thể chạy test bằng block assert bên dưới;
- Sau khi hoàn thành, đặt tên file theo mẫu sau: `labXX_thinking_<mssv>_<HoVaTen>.ipynb` (VD: *lab01_thinking_12345678_NguyenVanA.ipynb*);
- **Lưu ý:** Không thêm block mới, sửa tên đề bài hay block code assert, **nếu vi phạm sẽ coi như bài làm bị huỷ**.

## Problem 1 – Smart Device System

You are designing a small system to model **smart devices** in a house.

---

### 1. Base Class: `Device`

Create a class called `Device` that represents a generic device.

#### Attributes
- `name` (string): name of the device  
- `power` (bool): whether the device is on or off  
- `energy_usage` (float): energy consumption per hour (in watts)

#### Methods
- `turn_on()`: turns the device on  
- `turn_off()`: turns the device off  
- `toggle()`: switches the device state (on → off, off → on)  
- `is_on()`: returns `True` if the device is on, otherwise `False`

#### Dunder Methods
- `__str__`  
  - Human-readable description  
  - Example: `"Device Lamp (ON)"`
- `__repr__`  
  - Unambiguous representation  
  - Example: `"Device(name='Lamp', power=True, energy_usage=60.0)"`
- `__eq__`  
  - Two devices are equal if **name and energy_usage** are the same

---

### 2. Subclass: `SmartLight`

Create a subclass `SmartLight` that extends `Device`.

#### Additional Attributes
- `brightness` (int): value from `0` to `100`

#### Additional / Overridden Methods
- `set_brightness(value)`  
  - Clamp value to `[0, 100]`  
  - If brightness > 0, the light must be **ON**
- Override `turn_off()`  
  - Turns the device off  
  - Sets brightness to `0`
- Override `__str__`  
  - Include brightness information  
  - Example: `"SmartLight Bedroom (ON, 80%)"`

---

### 3. Operator Overloading
- Implement `__add__` for `Device`
- `device1 + device2` returns the **total energy usage** (float)
- Operation is valid only if both operands are `Device` instances

---

### Notes
- Do **not** worry about private vs public attributes
- Focus on:
  - class definition
  - initializer (`__init__`)
  - methods and behaviors
  - inheritance & overriding
  - dunder methods and operator overloading
- The problem is intentionally **basic but not trivial**

Implement the classes so that **all test cases pass**.

In [27]:
class Device:
    def __init__(self, name, energy_usage):
        self.name = name
        self.power = False
        self.energy_usage = energy_usage
    def turn_on(self):
        self.power = True

    def turn_off(self):
        self.power = False

    def toggle(self):
        self.power = not self.power

    def is_on(self):
        return self.power

    def __str__(self):
        if self.power:
            return f"Device '{self.name}' (ON)"
        else:
            return f"Device '{self.name}' (OFF)"

    def __repr__(self):
        return f"Device(name='{self.name}', power = {self.power}, energy_usage={self.energy_usage})"

    def __eq__(self, other):
        return self.name == other.name and self.energy_usage == other.energy_usage

    def __add__(self, other):
        if isinstance(other, Device):
            return self.energy_usage + other.energy_usage


class SmartLight(Device):
    def __init__(self, name, energy_usage, brightness):
        super().__init__(name, energy_usage)
        self.brightness = brightness
        if self.brightness > 0: self.power = True

    def set_brightness(self, value):
        self.brightness = max(0, min(100, value))
        if self.brightness > 0: self.power = True
        else: self.power = False

    def turn_off(self):
        self.brightness = 0
        self.power = False

    def __str__(self):
        status = "ON" if self.power else "OFF"
        return f"SmartLight {self.name} ({status}, {self.brightness}%)"
# print(d2.energy_usage)
light = SmartLight("Bedroom", 10.0, 50)
s = str(light)
print(s)

SmartLight Bedroom (ON, 50%)


In [28]:
# ===============================
# Base Device tests
# ===============================
d1 = Device("Lamp", 60.0)
d2 = Device("Lamp", 60.0)
d3 = Device("Fan", 75.0)

# --- Attribute existence ---
for attr in ["name", "power", "energy_usage"]:
    assert hasattr(d1, attr), f"Missing attribute: {attr}"

# --- Initial state ---
assert d1.name == "Lamp"
assert d1.energy_usage == 60.0
assert d1.power is False
assert d1.is_on() is False

# --- turn_on / turn_off ---
d1.turn_on()
assert d1.power is True
assert d1.is_on() is True

d1.turn_off()
assert d1.power is False
assert d1.is_on() is False

# --- toggle ---
d1.toggle()
assert d1.is_on() is True

d1.toggle()
assert d1.is_on() is False

# --- __eq__ ---
assert d1 == d2
assert not (d1 == d3)
assert d1 != d3

# --- __add__ ---
total_energy = d1 + d3
assert isinstance(total_energy, float)
assert total_energy == 135.0

# --- __str__ ---
s = str(d1)
assert isinstance(s, str)
assert "Lamp" in s
assert "OFF" in s or "ON" in s

# --- __repr__ ---
r = repr(d1)
assert isinstance(r, str)
assert "Device" in r
assert "name='Lamp'" in r
assert "energy_usage=60.0" in r


# ===============================
# SmartLight tests
# ===============================
light = SmartLight("Bedroom", 10.0, 50)

# --- Inheritance ---
assert isinstance(light, SmartLight)
assert isinstance(light, Device)

# --- Attribute existence ---
for attr in ["name", "power", "energy_usage", "brightness"]:
    assert hasattr(light, attr), f"Missing attribute: {attr}"

# --- Initial state ---
assert light.name == "Bedroom"
assert light.energy_usage == 10.0
assert light.brightness == 50
assert light.is_on() is True   # brightness > 0 implies ON

# --- set_brightness (clamp & side effect) ---
light.set_brightness(80)
assert light.brightness == 80
assert light.is_on() is True

light.set_brightness(150)
assert light.brightness == 100

light.set_brightness(-20)
assert light.brightness == 0

# --- turn_off override ---
light.set_brightness(40)
light.turn_off()

assert light.is_on() is False
assert light.brightness == 0

# --- __str__ override ---
s = str(light)
assert isinstance(s, str)
assert "SmartLight" in s
assert "Bedroom" in s
assert "%" in s

## Problem 2 – Smart Home Appliances (Advanced)

You are extending the smart home system to support **appliances with modes, comparison, and polymorphic behavior**.

This problem focuses on:
- deeper use of `__init__`
- method interaction & state consistency
- subclass-specific rules
- multiple dunder methods (`__eq__`, `__lt__`, `__str__`, `__repr__`)
- operator overloading with **semantic meaning**
- correctness under stricter test cases

---

### 1. Base Class: `Appliance`

Create a class `Appliance` representing a generic home appliance.

#### Attributes
- `name` (str): appliance name
- `power` (bool): whether the appliance is on
- `base_power` (float): base energy usage (watts)

#### Methods
- `turn_on()`
- `turn_off()`
- `is_on()`
- `current_power_usage()`  
  - returns `0.0` if OFF  
  - returns energy usage if ON (may be overridden)

#### Dunder Methods
- `__str__`
  - Example: `"Appliance Washer (ON)"`
- `__repr__`
  - Must include class name and all core attributes
- `__eq__`
  - Two appliances are equal if:
    - same class
    - same name
    - same base_power
- `__lt__`
  - Compare appliances by `current_power_usage`
- `__add__`
  - `a1 + a2` → total `current_power_usage`
  - Works only for `Appliance` instances

---

### 2. Subclass: `SmartWasher`

Represents a washing machine with washing modes.

#### Additional Attributes
- `mode` (str): one of `"eco"`, `"normal"`, `"heavy"`
- `mode_multiplier` (dict):
  - `"eco"` → `0.7`
  - `"normal"` → `1.0`
  - `"heavy"` → `1.5`

#### Methods
- `set_mode(mode)`
  - Invalid mode → raise `ValueError`
  - Changing mode does **not** auto turn on
- Override `current_power_usage()`
  - OFF → `0.0`
  - ON → `base_power * multiplier`
- Override `__str__`
  - Example: `"SmartWasher LG (ON, mode=eco)"`

---

### 3. Subclass: `SmartAC`

Represents an air conditioner.

#### Additional Attributes
- `temperature` (int): allowed range `[16, 30]`

#### Methods
- `set_temperature(temp)`
  - Clamp value to valid range
- Override `current_power_usage()`
  - OFF → `0.0`
  - ON → energy increases as temperature decreases  
  - Formula:  
    ```
    base_power * (1 + (30 - temperature) * 0.05)
    ```
- Override `__str__`
  - Example: `"SmartAC Daikin (ON, 22C)"`

---

### 4. Design Rules
- Do NOT use private attributes
- Do NOT hardcode behavior in test cases
- All subclasses must remain usable via `Appliance` references
- Focus on **correct state transitions**
- Prefer calling `super()` when appropriate

Your implementation must pass all test cases below.


In [29]:
class Appliance:
    def __init__(self, name, base_power):
        self.name = name
        self.base_power = float(base_power)
        self.power = False  

    def turn_on(self):
        self.power = True

    def turn_off(self):
        self.power = False

    def is_on(self):
        return self.power

    def current_power_usage(self):
        return self.base_power if self.power else 0.0

    def __str__(self):
        status = "ON" if self.power else "OFF"
        return f"Appliance {self.name} ({status})"

    def __repr__(self):
        return f"{self.__class__.__name__}(name='{self.name}', base_power={self.base_power}, power={self.power})"

    def __eq__(self, other):
        if not isinstance(other, Appliance):
            return False
        return (type(self) == type(other) and 
                self.name == other.name and 
                self.base_power == other.base_power)

    def __lt__(self, other):
        if not isinstance(other, Appliance):
            return NotImplemented
        return self.current_power_usage() < other.current_power_usage()

    def __add__(self, other):
        if not isinstance(other, Appliance):
            return NotImplemented
        return self.current_power_usage() + other.current_power_usage()


class SmartWasher(Appliance):
    def __init__(self, name, base_power, mode="normal"):
        super().__init__(name, base_power)
        self.mode_multiplier = {
            "eco": 0.7,
            "normal": 1.0,
            "heavy": 1.5
        }
        if mode not in self.mode_multiplier:
            raise ValueError(f"Invalid mode: {mode}")
        self.mode = mode

    def set_mode(self, mode):
        if mode not in self.mode_multiplier:
            raise ValueError(f"Invalid mode: {mode}")
        self.mode = mode

    def current_power_usage(self):
        if not self.power:
            return 0.0
        return self.base_power * self.mode_multiplier[self.mode]

    def __str__(self):
        status = "ON" if self.power else "OFF"
        return f"SmartWasher {self.name} ({status}, mode={self.mode})"


class SmartAC(Appliance):
    def __init__(self, name, base_power, temperature=24):
        super().__init__(name, base_power)
        self.temperature = 0 
        self.set_temperature(temperature) 

    def set_temperature(self, temperature):
        self.temperature = max(16, min(30, int(temperature)))

    def current_power_usage(self):
        if not self.power:
            return 0.0
 
        factor = 1 + (30 - self.temperature) * 0.05
        return self.base_power * factor

    def __str__(self):
        status = "ON" if self.power else "OFF"
        return f"SmartAC {self.name} ({status}, {self.temperature}C)"

In [30]:
# ===============================
# Appliance base tests
# ===============================
a1 = Appliance("Generic", 100.0)
a2 = Appliance("Generic", 100.0)
a3 = Appliance("Other", 150.0)

assert hasattr(a1, "name")
assert hasattr(a1, "power")
assert hasattr(a1, "base_power")

assert a1.is_on() is False
assert a1.current_power_usage() == 0.0

a1.turn_on()
assert a1.is_on() is True
assert a1.current_power_usage() == 100.0

a1.turn_off()
assert a1.current_power_usage() == 0.0

assert a1 == a2
assert a1 != a3

a1.turn_on()
a3.turn_on()
assert (a1 + a3) == 250.0

assert a1 < a3


# ===============================
# SmartWasher tests
# ===============================
washer = SmartWasher("LG", 500.0, mode="eco")

assert isinstance(washer, Appliance)
assert washer.is_on() is False
assert washer.mode == "eco"

washer.turn_on()
assert washer.current_power_usage() == 350.0

washer.set_mode("heavy")
assert washer.current_power_usage() == 750.0

try:
    washer.set_mode("invalid")
    assert False, "Expected ValueError"
except ValueError:
    pass

s = str(washer)
assert "SmartWasher" in s
assert "mode=heavy" in s


# ===============================
# SmartAC tests
# ===============================
ac = SmartAC("Daikin", 1000.0, temperature=24)

assert ac.is_on() is False
assert ac.current_power_usage() == 0.0

ac.turn_on()
assert ac.temperature == 24
assert ac.current_power_usage() == 1000.0 * (1 + (30 - 24) * 0.05)

ac.set_temperature(10)
assert ac.temperature == 16

ac.set_temperature(35)
assert ac.temperature == 30

s = str(ac)
assert "SmartAC" in s
assert "30C" in s


# ===============================
# Polymorphism & comparison
# ===============================
devices = [washer, ac]
for d in devices:
    assert isinstance(d, Appliance)

washer.turn_on()
ac.turn_on()

assert devices[0] < devices[1] or devices[1] < devices[0]


## Problem 3 – Smart Monitoring System (Multiple Inheritance & Composition)

In this problem, you will design a **smart monitoring system** that combines:

- **Multiple inheritance** (behavior + capability)
- **Composition** (an object *has another object*)
- Careful use of `super()`
- Consistent state management across classes
- Meaningful dunder methods

This problem is intentionally harder and closer to *real-world OOP design*.

---

### 1. Capability Class: `Connectable`

Represents the ability to connect to a network.

#### Attributes
- `connected` (bool)

#### Methods
- `connect()`: set connected to True
- `disconnect()`: set connected to False
- `is_connected()`: return connection state

---

### 2. Capability Class: `Measurable`

Represents the ability to measure something.

#### Attributes
- `last_value` (float or None)

#### Methods
- `measure()`
  - returns a float value
  - updates `last_value`
  - base class raises `NotImplementedError`

---

### 3. Component (Composition): `EnergyMeter`

This class is NOT an appliance itself.

#### Attributes
- `total_energy` (float): accumulated energy usage

#### Methods
- `record(amount)`
  - amount must be non-negative
  - adds amount to `total_energy`
- `reset()`

---

### 4. Base Class: `Device`

#### Attributes
- `name` (str)
- `power` (bool)

#### Methods
- `turn_on()`
- `turn_off()`
- `is_on()`

#### Dunder Methods
- `__str__`
- `__repr__`

---

### 5. Multiple Inheritance Class: `SmartSensor`

`SmartSensor` inherits from:
- `Device`
- `Connectable`
- `Measurable`

#### Additional Attributes
- `meter` (EnergyMeter): composed object

#### Methods
- `read()`
  - Only allowed when:
    - device is ON
    - device is connected
  - Calls `measure()`
  - Records the measured value into `meter`
  - Returns the measured value

#### Dunder Methods
- Override `__str__`
  - Example: `"SmartSensor TempSensor (ON, connected, total=12.5)"`

---

### 6. Concrete Sensor: `TemperatureSensor`

Subclass of `SmartSensor`.

#### Behavior
- Implements `measure()`
  - returns a float temperature value (hardcode or simple logic is fine)
- No random required; deterministic is acceptable

---

### Design Rules
- Use `super()` correctly in multiple inheritance
- Do NOT duplicate state across classes
- Composition (`EnergyMeter`) must be used, not inheritance
- Raise meaningful errors when rules are violated

Your implementation must satisfy all test cases below.


In [31]:
class Connectable:
    def __init__(self):
        self.connected = False

    def connect(self):
        self.connected = True

    def disconnect(self):
        self.connected = False

    def is_connected(self):
        return self.connected


class Measurable:
    def __init__(self):
        self.last_value = None

    def measure(self):
        raise NotImplementedError("Subclasses must implement measure()")


class EnergyMeter:
    def __init__(self):
        self.total_energy = 0.0

    def record(self, amount):
        if amount < 0:
            raise ValueError("Energy amount cannot be negative")
        self.total_energy += amount

    def reset(self):
        self.total_energy = 0.0


class Device:
    def __init__(self, name):
        self.name = name
        self.power = False

    def turn_on(self):
        self.power = True

    def turn_off(self):
        self.power = False

    def is_on(self):
        return self.power

    def __str__(self):
        state = "ON" if self.power else "OFF"
        return f"Device {self.name} ({state})"

    def __repr__(self):
        return f"Device(name='{self.name}')"


class SmartSensor(Device, Connectable, Measurable):
    def __init__(self, name):
        Device.__init__(self, name)
        Connectable.__init__(self)
        Measurable.__init__(self)
        self.meter = EnergyMeter()

    def read(self):
        if not self.is_on():
            raise RuntimeError(f"Cannot read {self.name}: Device is OFF")
        
        if not self.is_connected():
            raise RuntimeError(f"Cannot read {self.name}: Device is DISCONNECTED")

        value = self.measure()
 
        self.meter.record(value)
        
        return value

    def __str__(self):
        status = "ON" if self.is_on() else "OFF"
        conn = "connected" if self.is_connected() else "disconnected"
        return (f"SmartSensor {self.name} ({status}, {conn}, "
                f"total={self.meter.total_energy})")


class TemperatureSensor(SmartSensor):
    
    def measure(self):
        current_temp = 24.5
        self.last_value = current_temp
        return current_temp


In [32]:
# =============================
# TemperatureSensor tests
# =============================

sensor = TemperatureSensor("TempSensor")

# --- attribute existence ---
assert hasattr(sensor, "name")
assert hasattr(sensor, "power")
assert hasattr(sensor, "connected")
assert hasattr(sensor, "last_value")
assert hasattr(sensor, "meter")

# --- initial state ---
assert sensor.is_on() is False
assert sensor.is_connected() is False
assert sensor.meter.total_energy == 0.0

# --- invalid read ---
try:
    sensor.read()
    assert False, "Expected RuntimeError when OFF or disconnected"
except RuntimeError:
    pass

# --- valid read ---
sensor.turn_on()
sensor.connect()

value = sensor.read()
assert isinstance(value, float)
assert sensor.last_value == value
assert sensor.meter.total_energy == value

# --- multiple reads accumulate ---
value2 = sensor.read()
assert sensor.meter.total_energy == value + value2

# --- disconnect prevents read ---
sensor.disconnect()
try:
    sensor.read()
    assert False
except RuntimeError:
    pass

# --- string representation ---
s = str(sensor)
assert "SmartSensor" in s
assert "TempSensor" in s
